In [ ]:
from dotenv import load_dotenv
from langchain_community.utilities import ApifyWrapper
from langchain_core.documents import Document

In [ ]:
# Loading the API keys
load_dotenv("../.env")

True

In [ ]:
apify = ApifyWrapper()

In [ ]:
loader = apify.call_actor(
    actor_id="apify/website-content-crawler",
    run_input={
        "startUrls": [{"url": "https://cohere.com/llmu"}],
        "maxCrawlDepth": 2,
    },
    dataset_mapping_function=lambda dataset_item: Document(
        page_content=(
            dataset_item["text"] if dataset_item["text"] else "No content available"
        ),
        metadata={
            "title": dataset_item["metadata"]["title"],
            "source": dataset_item["url"],
        },
    ),
)

In [ ]:
loader

ApifyDatasetLoader(apify_client=<apify_client.client.ApifyClient object at 0x0000023B3EC5E230>, dataset_id='cq8YtWSRERjS0G9Qq', dataset_mapping_function=<function <lambda> at 0x0000023B3EC41990>)

In [ ]:
docs = loader.load()

In [ ]:
docs

[Document(metadata={'title': 'LLM University | Cohere', 'source': 'https://cohere.com/llmu'}, page_content='LLM University | CohereQualified\nWelcome to LLM University, your premier learning destination for mastering Enterprise AI technologies. Designed for developers and technical professionals, our hub offers comprehensive resources, expert-led courses, and step-by-step guides to help you start building quickly and stay ahead in the rapidly evolving AI landscape.\nJoin us to enhance your skills, drive innovation, and unlock the full potential of AI in your enterprise.'),
 Document(metadata={'title': 'Reranking', 'source': 'https://cohere.com/llmu/reranking'}, page_content="RerankingQualified\nThis chapter uses the same notebook as the previous chapter.\nIn previous chapters, you learned keyword search and dense retrieval, and you were able to apply them by querying a large Wikipedia article dataset. You noticed that keyword search performed well with some queries, and not so well wit

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
len(docs[0].page_content)

466

In [ ]:
len(docs[10].page_content)

10708

In [ ]:
len(docs)

51

In [ ]:
lengths = []
for doc in docs:
    x = len(doc.page_content)
    lengths.append(x)

In [ ]:
lengths

[466,
 4113,
 8898,
 21332,
 20412,
 10834,
 11794,
 16466,
 10873,
 31397,
 10708,
 15184,
 14255,
 9727,
 16093,
 21687,
 22435,
 12602,
 9143,
 6448,
 16075,
 10785,
 11162,
 12920,
 12701,
 10930,
 15367,
 24705,
 8766,
 15962,
 13198,
 6271,
 3163,
 11542,
 7033,
 5904,
 10372,
 19277,
 8207,
 6243,
 10894,
 11707,
 6131,
 10639,
 7842,
 6490,
 9152,
 17299,
 14546,
 6265,
 6943]

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000, chunk_overlap=200, length_function=len
)

In [ ]:
docs_split = text_splitter.split_documents(docs)

In [ ]:
docs_split

[Document(metadata={'title': 'LLM University | Cohere', 'source': 'https://cohere.com/llmu'}, page_content='LLM University | CohereQualified\nWelcome to LLM University, your premier learning destination for mastering Enterprise AI technologies. Designed for developers and technical professionals, our hub offers comprehensive resources, expert-led courses, and step-by-step guides to help you start building quickly and stay ahead in the rapidly evolving AI landscape.\nJoin us to enhance your skills, drive innovation, and unlock the full potential of AI in your enterprise.'),
 Document(metadata={'title': 'Reranking', 'source': 'https://cohere.com/llmu/reranking'}, page_content='RerankingQualified\nThis chapter uses the same notebook as the previous chapter.\nIn previous chapters, you learned keyword search and dense retrieval, and you were able to apply them by querying a large Wikipedia article dataset. You noticed that keyword search performed well with some queries, and not so well wit

In [ ]:
len(docs_split)

396

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.vectorstores import DeepLake
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore

d:\Softwares\anaconda\envs\Langchain-RAG\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [ ]:
embedding = CohereEmbeddings(model="embed-english-v3.0")

d:\Softwares\anaconda\envs\Langchain-RAG\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `CohereEmbeddings` was deprecated in LangChain 0.0.30 and will be removed in 0.3.0. An updated version of the class exists in the langchain-cohere package and should be used instead. To use it run `pip install -U langchain-cohere` and import as `from langchain_cohere import CohereEmbeddings`.
  warn_deprecated(
d:\Softwares\anaconda\envs\Langchain-RAG\lib\site-packages\pydantic\_internal\_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


KeyError: 'user_agent'

In [ ]:
import cohere

In [ ]:
import time

In [ ]:
DeepLake.force_delete_by_path(f"hub://{org_id}/{db_name}")

In [ ]:
embedding = CohereEmbeddings(model="embed-english-v3.0", user_agent="langchain")  # 1024

In [ ]:
# DeepLake Vector Store
org_id = "vijayv2807"
db_name = "cohere-llmu-data"
dataset_path = f"hub://{org_id}/{db_name}"
db = DeepLake(dataset_path=dataset_path, embedding=embedding)

Your Deep Lake dataset has been successfully created!


In [ ]:
db

In [ ]:
db.add_documents(docs_split)

Creating 396 embeddings in 1 batches of size 396:: 100%|██████████| 1/1 [01:37<00:00, 97.45s/it]

Dataset(path='hub://vijayv2807/cohere-llmu-data', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype       shape      dtype  compression
  -------    -------     -------    -------  ------- 
   text       text      (396, 1)      str     None   
 metadata     json      (396, 1)      str     None   
 embedding  embedding  (396, 1024)  float32   None   
    id        text      (396, 1)      str     None   


['94ab9cfd-5b83-11ef-8267-646c80368716',
 '94ab9cfe-5b83-11ef-9930-646c80368716',
 '94ab9cff-5b83-11ef-9713-646c80368716',
 '94ab9d00-5b83-11ef-b7f5-646c80368716',
 '94ab9d01-5b83-11ef-b9b3-646c80368716',
 '94ab9d02-5b83-11ef-bfaa-646c80368716',
 '94ab9d03-5b83-11ef-b8a5-646c80368716',
 '94ab9d04-5b83-11ef-a883-646c80368716',
 '94ab9d05-5b83-11ef-b2f6-646c80368716',
 '94ab9d06-5b83-11ef-99c5-646c80368716',
 '94ab9d07-5b83-11ef-9fad-646c80368716',
 '94ab9d08-5b83-11ef-aea1-646c80368716',
 '94ab9d09-5b83-11ef-855b-646c80368716',
 '94ab9d0a-5b83-11ef-a4ba-646c80368716',
 '94ab9d0b-5b83-11ef-81b3-646c80368716',
 '94ab9d0c-5b83-11ef-9be2-646c80368716',
 '94ab9d0d-5b83-11ef-bba8-646c80368716',
 '94ab9d0e-5b83-11ef-b0e0-646c80368716',
 '94ab9d0f-5b83-11ef-bfd2-646c80368716',
 '94ab9d10-5b83-11ef-a5be-646c80368716',
 '94ab9d11-5b83-11ef-b621-646c80368716',
 '94ab9d12-5b83-11ef-9599-646c80368716',
 '94ab9d13-5b83-11ef-86dd-646c80368716',
 '94ab9d14-5b83-11ef-a1e1-646c80368716',
 '94ab9d15-5b83-

In [ ]:
# Pinecone Vector Store
index_name = "cohere-llmu-data"
pc = Pinecone()

In [ ]:
existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

In [ ]:
existing_indexes

['ai-assistant', 'lang-sample-2', 'langchain-demo']

In [ ]:
if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=1024,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [ ]:
index

In [ ]:
vector_store = PineconeVectorStore(index=index, embedding=embedding)

In [ ]:
vector_store.add_documents(docs_split)

['a82f3981-af02-40a7-9eef-4665dba0f262',
 '7431654f-5db4-44a8-9b86-882f48d8786c',
 '9c6d9200-f171-4e81-9e2d-c8ac4f8e71f5',
 '1ed6ae39-ab3d-4990-ad2a-886be7ebd668',
 '48c88946-99ae-463a-9fd1-533b14699bd7',
 '7c77c8fa-ce78-4390-b074-2b07b0d63ee7',
 '20abf1ce-e808-4344-a7b8-f77d87f4982c',
 'b8174db5-86de-4cb8-8c48-21d63e5d6342',
 '261ea28c-131b-4234-96e5-aef418468d0b',
 '06ce9803-ecea-473e-8421-cb44f4cce324',
 'a505b81e-cc03-4423-9a8b-711ed63ea3b0',
 '50f71b5d-ed42-4e96-8853-ca79165860c8',
 '67842fdb-2d4a-4bf3-a3fb-ca8b17cb9d07',
 'd30baf30-544d-4f05-be3d-63a18ecd5f20',
 '3155bac2-70b3-495e-94a9-3d0dff3cde13',
 '1edddef1-ff47-45a6-9b29-963c894fb83d',
 '63b1c7f1-21e5-45a3-bc71-96579f9ecc68',
 '8b824615-a457-421f-872b-d0ba0e0023e8',
 '888d46cc-70ef-439a-a534-df5be0f03193',
 'f562eada-36fe-407c-9f82-90db8668590e',
 '125ec80d-c33c-4094-bcce-3d86dbc2f91d',
 '8b1c9922-6925-4956-92d8-bfec306e5dd1',
 '59c99c8b-ed22-4c92-afdf-84cfbd0550af',
 '91cddbe1-0a9a-4f51-af0c-5d245906f7d6',
 '44fb4fc8-d44f-